# Student Loan Risk with Deep Learning

In [1]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [2]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [3]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [4]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [6]:
# Define the target set y using the credit_ranking column
y = loans_df['credit_ranking']

# Display a sample of y
y.head()

0    0
1    0
2    0
3    1
4    0
Name: credit_ranking, dtype: int64

In [8]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop('credit_ranking', axis=1)

# Review the features DataFrame
X.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [9]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [11]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = scaler.transform(X_train)

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [13]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.columns)

# Review the number of features
print(f"Number of Features: {number_input_features}")

Number of Features: 11


In [16]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 = 80

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = 30

# Define the number of neurons in the output layer
output_nodes = 1

In [20]:
# Create the Sequential model instance
model = tf.keras.models.Sequential()

# Add the first hidden layer
model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Add the second hidden layer
model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation functio
model.add(tf.keras.layers.Dense(units=output_nodes, activation="sigmoid"))


In [21]:
# Display the Sequential model summary
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                960       
                                                                 
 dense_4 (Dense)             (None, 30)                2430      
                                                                 
 dense_5 (Dense)             (None, 1)                 31        
                                                                 
Total params: 3421 (13.36 KB)
Trainable params: 3421 (13.36 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [22]:
# Compile the Sequential model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [23]:
# Fit the model using 50 epochs and the training data
model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
38/38 [==============================] - 2s 9ms/step - loss: 0.6336 - accuracy: 0.6597
Epoch 2/50
38/38 [==============================] - 0s 10ms/step - loss: 0.5394 - accuracy: 0.7506
Epoch 3/50
38/38 [==============================] - 0s 5ms/step - loss: 0.5105 - accuracy: 0.7581
Epoch 4/50
38/38 [==============================] - 0s 5ms/step - loss: 0.4962 - accuracy: 0.7623
Epoch 5/50
38/38 [==============================] - 0s 8ms/step - loss: 0.4851 - accuracy: 0.7623
Epoch 6/50
38/38 [==============================] - 0s 10ms/step - loss: 0.4811 - accuracy: 0.7665
Epoch 7/50
38/38 [==============================] - 0s 5ms/step - loss: 0.4714 - accuracy: 0.7798
Epoch 8/50
38/38 [==============================] - 0s 5ms/step - loss: 0.4672 - accuracy: 0.7765
Epoch 9/50
38/38 [==============================] - 0s 4ms/step - loss: 0.4623 - accuracy: 0.7781
Epoch 10/50
38/38 [==============================] - 0s 5ms/step - loss: 0.4558 - accuracy: 0.7848
Epoch 11/50
38/38

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [27]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
loss, accuracy = model.evaluate(X_test, y_test)

# Display the model loss and accuracy results
print(f"Test loss: {loss}")
print(f"Test accuracy: {accuracy}")

13/13 [==============================] - 0s 8ms/step - loss: 16.4410 - accuracy: 0.5350
Test loss: 16.440950393676758
Test accuracy: 0.5350000262260437


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [28]:
# Set the model's file path
model_file_path = "/content/drive/MyDrive/Colab Notebooks/credit_risk_model.h5"

# Export your model to a keras file
model.save(model_file_path)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [29]:
# Set the model's file path
model_file_path = "/content/drive/MyDrive/Colab Notebooks/credit_risk_model.h5"

# Load the model to a new object
loaded_model = tf.keras.models.load_model(model_file_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [31]:
# Make predictions with the test data
predictions = loaded_model.predict(X_test)

# Display a sample of the predictions
predictions[:5]


13/13 [==============================] - 0s 7ms/step


array([[3.3479434e-17],
       [6.3327372e-32],
       [9.9680543e-01],
       [1.3623012e-19],
       [9.9840534e-01]], dtype=float32)

In [32]:
# Save the predictions to a DataFrame and round the predictions to binary results
pred_df = pd.DataFrame(predictions, columns=["Probability"])
pred_df['Prediction'] = pred_df['Probability'].round()
pred_df.head()


,Probability,Prediction
0,3.347943e-17,0.0
1,6.332737e-32,0.0
2,9.968054e-01,1.0
3,1.362301e-19,0.0
4,9.984053e-01,1.0


### Step 4: Display a classification report with the y test data and predictions

In [33]:
# Print the classification report with the y test data and predictions
from sklearn.metrics import classification_report

print(classification_report(y_test, pred_df['Prediction']))

              precision    recall  f1-score   support

           0       0.50      0.84      0.63       188
           1       0.65      0.27      0.38       212

    accuracy                           0.54       400
   macro avg       0.58      0.55      0.50       400
weighted avg       0.58      0.54      0.50       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.
  

*  Student Demographics: Help understand the needs of different student groups.
* Student financial Information: Understand a students ability to repay loan and financial need for assistance.
* Student Academic information: Understand a student's potential for success and ability to find employment post school
* Student Loan options: Helps compare different loan options and recommend the best options for each student


2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

**Context Based filtering** because it provides information about both the students and the loan options

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

*   Limited data and information on student. Not have enough data to make accurate recommendations for all students
*   Fairness & Bias: Recommendation systems must be fair and unbiased. This means that they must not discriminate against students based on their race, ethnicity, gender, or other protected characteristics.



**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**


**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**


**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

